# Base.convert to String ambiguous issue

In [ ]:
using Dates

In [ ]:
import Base.convert

In [ ]:
Base.convert(String, date::DateTime) = "$date"

In [ ]:
convert

In [ ]:
convert

In [ ]:
struct MyStruct
    a::Integer
end

In [ ]:
MyStruct(1)

In [ ]:
# CAUTION this is WRONG, this is causing IJulia to restart
Base.convert(String, x::MyStruct) = "$x.a"

In [ ]:
# I should be using ::Type{String} instead!
Base.convert(::Type{String}, x::MyStruct) = "$(x.a)"

In [ ]:
Base.convert(String, MyStruct(1))

In [ ]:
String

In [ ]:
::Type{String}

In [ ]:
Base.convert

# GPU incompatibility

In [ ]:
using CUDA

In [ ]:
logistic(x) = 1 / (1 + exp(-x))

In [ ]:
A = cu(rand(5, 10))

In [ ]:
logistic.(A)

# Clean up saved models

In [ ]:
include("exp.jl")

In [ ]:
for dir in readdir_nohidden("saved_models")
    keep_most_recent!(joinpath("saved_models", dir))
end

# CUDA memory limit

In [ ]:
# setup GPU memory limit

# For 1070
g=5.0
# For 2080 Ti
# g=9.0

ENV["JULIA_CUDA_MEMORY_LIMIT"] = convert(Int, round(g * 1024 * 1024 * 1024))

# Model size

In [ ]:
import Printf

In [ ]:
function test_size()
    # model size
    @info "FC model"
    for d in [7, 10,15,20,25,30]
        Printf.@printf "%.2f\n" param_count(fc_model_fn(d)) / 1e6
    end
    @info "FC deep model"
    for d in [7, 10,15,20,25,30]
        Printf.@printf "%.2f\n" param_count(deep_fc_model_fn(d)) / 1e6
    end
    # EQ models is independent of input size
    @info "EQ model"
    Printf.@printf "%.2f\n" param_count(eq_model_fn(10)) / 1e6
    Printf.@printf "%.2f\n" param_count(deep_eq_model_fn(10)) / 1e6
end

# Train ensembly with different K

- because K is probably the most important hyper-parameter for real data
- [X] the data could be loaded more effectively, e.g. inside one dataset `ds`. Then the data can be completely mixed.

experiments:
- [X] try ER/SF graphs
- [ ] try COR mat
- [-] try normalized COV (medCOV and maxCOV)
- [ ] try COR + VAR as input
- [X] try similar k (1,2,3,4]: this works
- [X] try larger batch UPDATE but 100 should be large enough

In [ ]:
include("exp.jl")

In [ ]:
# first try to use multiple dses for training
specs = []
for k in [1,5,10,20]
    push!(specs, DataSpec(d=11, k=k, gtype=:SF, noise=:Gaussian, mat=:maxCOV))
end
specs = Array{DataSpec}(specs)

In [ ]:
# train
expID = exp_train(specs, deep_eq_model_fn, prefix="ensK", train_steps=1e4)

In [ ]:
expID = exp_train(specs, deep_eq_model_fn, prefix="ensK-$(now())", train_steps=1e4, merge=true)

In [ ]:
# or I could train with K specifically designed for Sachs-2005


In [ ]:
# construct many dses
ds, test_ds = spec2ds(specs)

In [ ]:
# merge data
function merge_dses(dses)
    # merge datasets
    # 1. merge raw_x and raw_y
    raw_x = cat([ds.raw_x for ds in dses]..., dims=3)
    raw_y = cat([ds.raw_y for ds in dses]..., dims=3)
    # 2. assert batch_size
    batch_size = dses[1].batch_size
    # 4. construct new ds
    DataSetIterator(raw_x, raw_y, batch_size)
end

In [ ]:
size(ds[1].raw_x)

In [ ]:
ds

In [ ]:
merge_dses(ds)

# generate k to match test data

In [ ]:
# the test data k=20
spec20 = DataSpec(d=10, k=21, gtype=:SF,
        noise=:Gaussian, mechanism=:Linear, mat=:COV)

In [ ]:
ds, test_ds = spec2ds(spec20)

In [ ]:
ds = load_sup_ds(spec20, spec20.bsize, use_raw=true)

In [ ]:
x, y = next_batch!(ds)

In [ ]:
size(x)

In [ ]:
size(y)

In [ ]:
ENV["COLUMNS"] = 200

In [ ]:
var(x[:,:,1], dims=1)

In [ ]:
var(var(x[:,:,1], dims=1))

In [ ]:
mean(var(x[:,:,1], dims=1))

In [ ]:
var(x[:,:,1])

In [ ]:
# print mean and var of different ks

In [ ]:
specs = map([1,2,5,10,15,20]) do k
    DataSpec(d=10, k=k, gtype=:SF,
        noise=:Gaussian, mechanism=:Linear, mat=:COV)
end

In [ ]:
function featurize_spec(spec)
    ds = load_sup_ds(spec20, spec20.bsize, use_raw=true)
    x, y = next_batch!(ds)
    mean1 = mean(var(x[:,:,1], dims=1))
    var1 = var(var(x[:,:,1], dims=1))
    # also the other data points. Maybe report 1:10
    for i in 1:10
        meani = mean(var(x[:,:,i], dims=1))
        vari = var(var(x[:,:,i], dims=1))
        @show meani, vari
    end
    return mean1, var1
end

In [ ]:
featurize_spec(specs[1])

In [ ]:
for spec in specs
    @show featurize_spec(spec)
end

In [ ]:
# generate k so that the variance matches var1
# Is there a closed-form solution?
#
# I need to calculate the likelihood
# of the test variance belong to the data generated with k

## train a network to predict the generating parameter k

- from COV, using CNN
- from Var, using FC

In [ ]:
include("exp.jl")

In [ ]:
# 1. generate data
# this is a regression problem
ks = 1:20
specs = map(ks) do k
    DataSpec(d=10, k=k, gtype=:SF,
        noise=:Gaussian, mechanism=:Linear, mat=:COV)
end

In [ ]:
# test
ds, test_ds = spec2ds(specs[1])

In [ ]:
# test
x, y = next_batch!(ds)
size(x), size(y)

In [ ]:
function cov2diag(COV)
    v = map(1:size(COV)[end]) do i
#         diag(COV[:,:,1,i])
#         LinearAlgebra.eigen(COV[:,:,1,i]).values
        LinearAlgebra.eigen(COV[:,:,1,i]).vectors[:, end]
    end
    return cat(v..., dims=2)
end

In [ ]:
function ds2kds(ds, k)
    # replace y with (1,128, the value is k)
    DataSetIterator(
        ds.raw_x,
#         cov2diag(ds.raw_x),
    #     ds.raw_y,
        ones(1, size(ds.raw_x)[end]) * k,
        1, 
        ds.batch_size, 
        ds.nbatch)
end

In [ ]:
# test
ds2kds(ds, 2)

In [ ]:
dses = map(specs, ks) do spec, k
    ds, _ = spec2ds(spec)
    ds2kds(ds, k)
end

In [ ]:
dses[8].raw_y

In [ ]:
kds = merge_dses(dses)

In [ ]:
x, y = next_batch!(kds)
y

In [ ]:
model = Chain(
          Conv((3,3), 1=>32, relu, pad=(1,1), stride=2),
#           BatchNorm(32),
          Conv((3,3), 32=>32, relu, pad=(1,1), stride=1),
#           BatchNorm(32),
          Conv((3,3), 32=>32, relu, pad=(1,1), stride=2),
#           BatchNorm(32),
          # CAUTION no relu in the last layer
          Conv((3,3), 32=>1),
          Flatten()) |> gpu

In [ ]:
model = Chain(
#     Dense(10,1024, relu),
#     Dense(1024, 1024, relu),
    Dense(10,1)) |> gpu

In [ ]:
x

In [ ]:
cov2diag(x)

In [ ]:
model(gpu(cov2diag(x)))

In [ ]:
cat(map(1:128) do i
    diag(x[:,:,1,i])
        end..., dims=2)

In [ ]:
print_cb = Flux.throttle((loss)->begin
                @show loss
            end, 2)

In [ ]:
function just_train(model, kds)
    # just train
    opt = ADAM(1e-6)
    ps = Flux.params(model)
    loss_metric = MeanMetric{Float64}()
    @showprogress 0.1 "Training.." for step in 1:10000
        x, y = next_batch!(kds)
        x = cov2diag(x)
        x, y = (x, y) |> gpu
        gs = gradient(ps) do
            out = model(x)
            loss = Flux.mse(out, y)
            add!(loss_metric, loss)
            loss
        end
        print_cb(get!(loss_metric))
        Flux.Optimise.update!(opt, ps, gs)
    end
end

In [ ]:
just_train(model, kds)

In [ ]:
model(gpu(cov2diag(x)))

In [ ]:
y

In [ ]:
Flux.mse(cpu(model(gpu(x))), y)

## Inspection

In [ ]:
using Plots

In [ ]:
using LinearAlgebra

In [ ]:
ENV["COLUMNS"] = 200

In [ ]:
dses[1]

In [ ]:
dses[10]

In [ ]:
x1, y1 = dses[1].raw_x, dses[1].raw_y
x10, y10 = dses[10].raw_x, dses[10].raw_y
x20, y20 = dses[20].raw_x, dses[20].raw_y;

In [ ]:
x1, y1 = next_batch!(dses[1])
size(x1), size(y1)

In [ ]:
x10, y10 = next_batch!(dses[10])
size(x10), size(y10)

In [ ]:
x20, y20 = next_batch!(dses[20])
size(x20), size(y20)

In [ ]:
x1[:,:,:,1]

In [ ]:
Diagonal(x1[:,:,1,1]) * ones(10)

In [ ]:
diag(x1[:,:,1,1])

In [ ]:
y1, y10

In [ ]:
mean(x1)

In [ ]:
mean(x10)

In [ ]:
mean(x20)

In [ ]:
LinearAlgebra.eigen(x1[:,:,1,1]).vectors[:, end]

In [ ]:
p1 = plot(map(1:128) do i
    maximum(LinearAlgebra.eigen(x1[:,:,1,i]).values)
        end)

In [ ]:
p10 = plot(map(1:128) do i
    maximum(LinearAlgebra.eigen(x10[:,:,1,i]).values)
        end)

In [ ]:
p20 = plot(map(1:128) do i
    maximum(LinearAlgebra.eigen(x20[:,:,1,i]).values)
        end)

In [ ]:
plot(p1, p10, p20, layout=(1,3))

In [ ]:
LinearAlgebra.eigen(x10[:,:,1,10])

In [ ]:
maximum(x1[:,:,:,:])

In [ ]:
y1[:,1]

In [ ]:
x10[:,:,:,1]

In [ ]:
maximum(x10[:,:,:,:])

In [ ]:
y10[:,1]

In [ ]:
size(dses[1].raw_x)

# Some rather random testing

In [ ]:
exp_train(DataSpec(d=10, k=1, gtype=:SF,
        noise=:Gaussian, mechanism=:Linear, mat=:COV),
    deep_eq_model_fn, prefix="test-EQ", train_steps=1e4)

In [ ]:
exp_train(DataSpec(d=10, k=1, gtype=:ER,
        noise=:Gaussian, mechanism=:Linear, mat=:COV),
    deep_eq_model_fn, prefix="test-EQ", train_steps=1e4)

In [ ]:
exp_train(DataSpec(d=20, k=1, gtype=:SF,
        noise=:Gaussian, mechanism=:Linear, mat=:COV),
    deep_eq_model_fn, prefix="test-EQ", train_steps=1e4)

In [ ]:
# Test the model
spec = DataSpec(d=10, k=1, gtype=:SF,
        noise=:Gaussian, mechanism=:Linear, mat=:COV)
expID = "EQ-$(dataspec_to_id(spec))"

In [ ]:
exp_test(expID, spec, use_raw=true)

In [ ]:
spec

In [ ]:
function adhoc_test(expID, spec)
    model, _ = joinpath("saved_models", expID) |> load_most_recent
    model = gpu(model)
    ds, test_ds = spec2ds(spec)
    metrics, t = sup_test_raw(model, test_ds, nbatch=16)
    return metrics
end

In [ ]:
adhoc_test(expID, spec)

In [ ]:
dataspec_to_id(spec)

In [ ]:
_results[expID=>"raw-$(dataspec_to_id(spec))"]